In [3]:
import time
import cv2
import glob
import os
import numpy as np
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.svm import LinearSVC

print(cv2.__version__)

4.5.2


In [1]:
'''
Load Dataset
'''
def scene15():

    train_folders = glob.glob("SCENE-15/train/*")
    train_folders.sort()
    classes = dict()
    x_train = list()
    y_train = list()
    for index, folder in enumerate(train_folders):
        label = os.path.basename(folder) #return the base name of pathname  "folder"
        classes[label] = index
        paths = glob.glob(os.path.join(folder, "*"))
        for path in paths:
            x_train.append(cv2.imread(path, 0))
            y_train.append(index)

    x_test = list()
    y_test = list()
    test_folders = glob.glob("SCENE-15/test/*")
    test_folders.sort()
    for folder in test_folders:
        label = os.path.basename(folder)
        index = classes[label]
        paths = glob.glob(os.path.join(folder, "*"))
        for path in paths:
            x_test.append(cv2.imread(path, 0))
            y_test.append(index)

    return x_train, y_train, x_test, y_test, sorted(classes.keys())

In [4]:
x_train, y_train, x_test, y_test, labels_names = scene15()

random_indices = list(range(len(y_train)))
random.shuffle(random_indices)
x_train = np.array(x_train)[random_indices].tolist() #여기다가 dtype=object?
y_train = np.array(y_train)[random_indices].tolist()

<ipython-input-4-421ad6419253>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train = np.array(x_train)[random_indices].tolist()


In [6]:
print(x_train[0])

[[168 158 145 ... 204 175 204]
 [153 168 164 ... 219 217 211]
 [133 173 172 ... 164 152 225]
 ...
 [103 155 130 ... 124 117 138]
 [127 118 111 ... 111 116 129]
 [127 104 146 ... 117 132 139]]


In [11]:
'''
Parameters
'''
patch_stride = 16
K = 20

In [12]:
'''
Extract Patches
'''
train_key_points = list()
train_feature_shapes = list()
for image in x_train:
    h, w = image.shape
    image_key_points = list()
    for x in range(0, w, patch_stride):
        for y in range(0, h, patch_stride):
            image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
    train_key_points.append(image_key_points)
    train_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))

In [13]:
test_key_points = list()
test_feature_shapes = list()

for image in x_test:
    h, w = image.shape
    image_key_points = list()
    for x in range(0, w, patch_stride):
        for y in range(0, h, patch_stride):
            image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
    test_key_points.append(image_key_points)
    test_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))

In [14]:
print(train_key_points[0])
print(test_key_points[0])

[<KeyPoint 00000175D1FBCC90>, <KeyPoint 00000175D1FBC990>, <KeyPoint 00000175D1FBCC00>, <KeyPoint 00000175D1FBC750>, <KeyPoint 00000175D1FBCC30>, <KeyPoint 00000175D1FBCCF0>, <KeyPoint 00000175D1FBCD20>, <KeyPoint 00000175D1FBCD50>, <KeyPoint 00000175D1FBCD80>, <KeyPoint 00000175D1FBCDB0>, <KeyPoint 00000175D1FBCDE0>, <KeyPoint 00000175D1FBCE10>, <KeyPoint 00000175D1FBCE70>, <KeyPoint 00000175D1FBCED0>, <KeyPoint 00000175D1FBCF90>, <KeyPoint 00000175D1FBCFC0>, <KeyPoint 00000175D1FBCF00>, <KeyPoint 00000175D1FBCEA0>, <KeyPoint 00000175D1FBCF60>, <KeyPoint 00000175DEDE6030>, <KeyPoint 00000175DEDE6060>, <KeyPoint 00000175DEDE6090>, <KeyPoint 00000175DEDE60C0>, <KeyPoint 00000175DEDE60F0>, <KeyPoint 00000175DEDE6120>, <KeyPoint 00000175DEDE6150>, <KeyPoint 00000175DEDE6180>, <KeyPoint 00000175DEDE61B0>, <KeyPoint 00000175DEDE61E0>, <KeyPoint 00000175DEDE6210>, <KeyPoint 00000175DEDE6240>, <KeyPoint 00000175DEDE6270>, <KeyPoint 00000175DEDE62A0>, <KeyPoint 00000175DEDE62D0>, <KeyPoint 000

[문제 1. 특징 추출 – 15 pts]  

1-1. 주어진 Keypoint를 통하여, 각각의 이미지마다 특징을 추출하시오.  
이 때, OpenCV 패키지를 활용하여, SIFT와 같은 특징을 추출하시오. (자세한 사항은 템플릿 코드를 참고)  

In [ ]:
'''
Extract Features
'''
# 아래의 코드의 빈 곳(None 부분)을 채우세요.
# None 부분 외의 부분은 가급적 수정 하지 말고, 주어진 형식에 맞추어
# None 부분 만을 채워주세요. 임의적으로 전체적인 구조를 수정하셔도 좋지만,
# 파이썬 코딩에 익숙 하지 않으시면, 가급적 틀을 유지하시는 것을 권장합니다.

# 1) descriptor를 선정하세요. (SIFT, SURF 등) OpenCV의 패키지를 사용하시면 됩니다.
# 2) for 반복문 안에서, 1)에서 정의한 descriptor를 통하여 features를 추출하세요.
#    features의 차원은 (# of keypoints, feature_dim) 입니다.

In [ ]:
descriptor = cv2.SIFT_create()

In [19]:
train_features = []
index = 0

for image, key_points in zip(x_train, train_key_points):
    _, features = descriptor.detectAndCompute(image, None)
    train_features.append(features)
    index += 1
    print("Extract Train Features ... {:4d}/{:4d}".format(index, len(x_train)))

Extract Train Features ...    1/1538
Extract Train Features ...    2/1538
Extract Train Features ...    3/1538
Extract Train Features ...    4/1538
Extract Train Features ...    5/1538
Extract Train Features ...    6/1538
Extract Train Features ...    7/1538
Extract Train Features ...    8/1538
Extract Train Features ...    9/1538
Extract Train Features ...   10/1538
Extract Train Features ...   11/1538
Extract Train Features ...   12/1538
Extract Train Features ...   13/1538
Extract Train Features ...   14/1538
Extract Train Features ...   15/1538
Extract Train Features ...   16/1538
Extract Train Features ...   17/1538
Extract Train Features ...   18/1538
Extract Train Features ...   19/1538
Extract Train Features ...   20/1538
Extract Train Features ...   21/1538
Extract Train Features ...   22/1538
Extract Train Features ...   23/1538
Extract Train Features ...   24/1538
Extract Train Features ...   25/1538
Extract Train Features ...   26/1538
Extract Train Features ...   27/1538
E

Extract Train Features ...  233/1538
Extract Train Features ...  234/1538
Extract Train Features ...  235/1538
Extract Train Features ...  236/1538
Extract Train Features ...  237/1538
Extract Train Features ...  238/1538
Extract Train Features ...  239/1538
Extract Train Features ...  240/1538
Extract Train Features ...  241/1538
Extract Train Features ...  242/1538
Extract Train Features ...  243/1538
Extract Train Features ...  244/1538
Extract Train Features ...  245/1538
Extract Train Features ...  246/1538
Extract Train Features ...  247/1538
Extract Train Features ...  248/1538
Extract Train Features ...  249/1538
Extract Train Features ...  250/1538
Extract Train Features ...  251/1538
Extract Train Features ...  252/1538
Extract Train Features ...  253/1538
Extract Train Features ...  254/1538
Extract Train Features ...  255/1538
Extract Train Features ...  256/1538
Extract Train Features ...  257/1538
Extract Train Features ...  258/1538
Extract Train Features ...  259/1538
E

Extract Train Features ...  467/1538
Extract Train Features ...  468/1538
Extract Train Features ...  469/1538
Extract Train Features ...  470/1538
Extract Train Features ...  471/1538
Extract Train Features ...  472/1538
Extract Train Features ...  473/1538
Extract Train Features ...  474/1538
Extract Train Features ...  475/1538
Extract Train Features ...  476/1538
Extract Train Features ...  477/1538
Extract Train Features ...  478/1538
Extract Train Features ...  479/1538
Extract Train Features ...  480/1538
Extract Train Features ...  481/1538
Extract Train Features ...  482/1538
Extract Train Features ...  483/1538
Extract Train Features ...  484/1538
Extract Train Features ...  485/1538
Extract Train Features ...  486/1538
Extract Train Features ...  487/1538
Extract Train Features ...  488/1538
Extract Train Features ...  489/1538
Extract Train Features ...  490/1538
Extract Train Features ...  491/1538
Extract Train Features ...  492/1538
Extract Train Features ...  493/1538
E

Extract Train Features ...  690/1538
Extract Train Features ...  691/1538
Extract Train Features ...  692/1538
Extract Train Features ...  693/1538
Extract Train Features ...  694/1538
Extract Train Features ...  695/1538
Extract Train Features ...  696/1538
Extract Train Features ...  697/1538
Extract Train Features ...  698/1538
Extract Train Features ...  699/1538
Extract Train Features ...  700/1538
Extract Train Features ...  701/1538
Extract Train Features ...  702/1538
Extract Train Features ...  703/1538
Extract Train Features ...  704/1538
Extract Train Features ...  705/1538
Extract Train Features ...  706/1538
Extract Train Features ...  707/1538
Extract Train Features ...  708/1538
Extract Train Features ...  709/1538
Extract Train Features ...  710/1538
Extract Train Features ...  711/1538
Extract Train Features ...  712/1538
Extract Train Features ...  713/1538
Extract Train Features ...  714/1538
Extract Train Features ...  715/1538
Extract Train Features ...  716/1538
E

Extract Train Features ...  922/1538
Extract Train Features ...  923/1538
Extract Train Features ...  924/1538
Extract Train Features ...  925/1538
Extract Train Features ...  926/1538
Extract Train Features ...  927/1538
Extract Train Features ...  928/1538
Extract Train Features ...  929/1538
Extract Train Features ...  930/1538
Extract Train Features ...  931/1538
Extract Train Features ...  932/1538
Extract Train Features ...  933/1538
Extract Train Features ...  934/1538
Extract Train Features ...  935/1538
Extract Train Features ...  936/1538
Extract Train Features ...  937/1538
Extract Train Features ...  938/1538
Extract Train Features ...  939/1538
Extract Train Features ...  940/1538
Extract Train Features ...  941/1538
Extract Train Features ...  942/1538
Extract Train Features ...  943/1538
Extract Train Features ...  944/1538
Extract Train Features ...  945/1538
Extract Train Features ...  946/1538
Extract Train Features ...  947/1538
Extract Train Features ...  948/1538
E

Extract Train Features ... 1151/1538
Extract Train Features ... 1152/1538
Extract Train Features ... 1153/1538
Extract Train Features ... 1154/1538
Extract Train Features ... 1155/1538
Extract Train Features ... 1156/1538
Extract Train Features ... 1157/1538
Extract Train Features ... 1158/1538
Extract Train Features ... 1159/1538
Extract Train Features ... 1160/1538
Extract Train Features ... 1161/1538
Extract Train Features ... 1162/1538
Extract Train Features ... 1163/1538
Extract Train Features ... 1164/1538
Extract Train Features ... 1165/1538
Extract Train Features ... 1166/1538
Extract Train Features ... 1167/1538
Extract Train Features ... 1168/1538
Extract Train Features ... 1169/1538
Extract Train Features ... 1170/1538
Extract Train Features ... 1171/1538
Extract Train Features ... 1172/1538
Extract Train Features ... 1173/1538
Extract Train Features ... 1174/1538
Extract Train Features ... 1175/1538
Extract Train Features ... 1176/1538
Extract Train Features ... 1177/1538
E

Extract Train Features ... 1375/1538
Extract Train Features ... 1376/1538
Extract Train Features ... 1377/1538
Extract Train Features ... 1378/1538
Extract Train Features ... 1379/1538
Extract Train Features ... 1380/1538
Extract Train Features ... 1381/1538
Extract Train Features ... 1382/1538
Extract Train Features ... 1383/1538
Extract Train Features ... 1384/1538
Extract Train Features ... 1385/1538
Extract Train Features ... 1386/1538
Extract Train Features ... 1387/1538
Extract Train Features ... 1388/1538
Extract Train Features ... 1389/1538
Extract Train Features ... 1390/1538
Extract Train Features ... 1391/1538
Extract Train Features ... 1392/1538
Extract Train Features ... 1393/1538
Extract Train Features ... 1394/1538
Extract Train Features ... 1395/1538
Extract Train Features ... 1396/1538
Extract Train Features ... 1397/1538
Extract Train Features ... 1398/1538
Extract Train Features ... 1399/1538
Extract Train Features ... 1400/1538
Extract Train Features ... 1401/1538
E

In [20]:
test_features = []
index = 0

for image, key_points in zip(x_test, test_key_points):
    _, features = descriptor.detectAndCompute(image, None)
    test_features.append(features)
    index += 1
    print("Extract Test Features ... {:4d}/{:4d}".format(index, len(x_test)))

Extract Test Features ...    1/1509
Extract Test Features ...    2/1509
Extract Test Features ...    3/1509
Extract Test Features ...    4/1509
Extract Test Features ...    5/1509
Extract Test Features ...    6/1509
Extract Test Features ...    7/1509
Extract Test Features ...    8/1509
Extract Test Features ...    9/1509
Extract Test Features ...   10/1509
Extract Test Features ...   11/1509
Extract Test Features ...   12/1509
Extract Test Features ...   13/1509
Extract Test Features ...   14/1509
Extract Test Features ...   15/1509
Extract Test Features ...   16/1509
Extract Test Features ...   17/1509
Extract Test Features ...   18/1509
Extract Test Features ...   19/1509
Extract Test Features ...   20/1509
Extract Test Features ...   21/1509
Extract Test Features ...   22/1509
Extract Test Features ...   23/1509
Extract Test Features ...   24/1509
Extract Test Features ...   25/1509
Extract Test Features ...   26/1509
Extract Test Features ...   27/1509
Extract Test Features ...   

Extract Test Features ...  229/1509
Extract Test Features ...  230/1509
Extract Test Features ...  231/1509
Extract Test Features ...  232/1509
Extract Test Features ...  233/1509
Extract Test Features ...  234/1509
Extract Test Features ...  235/1509
Extract Test Features ...  236/1509
Extract Test Features ...  237/1509
Extract Test Features ...  238/1509
Extract Test Features ...  239/1509
Extract Test Features ...  240/1509
Extract Test Features ...  241/1509
Extract Test Features ...  242/1509
Extract Test Features ...  243/1509
Extract Test Features ...  244/1509
Extract Test Features ...  245/1509
Extract Test Features ...  246/1509
Extract Test Features ...  247/1509
Extract Test Features ...  248/1509
Extract Test Features ...  249/1509
Extract Test Features ...  250/1509
Extract Test Features ...  251/1509
Extract Test Features ...  252/1509
Extract Test Features ...  253/1509
Extract Test Features ...  254/1509
Extract Test Features ...  255/1509
Extract Test Features ...  2

Extract Test Features ...  463/1509
Extract Test Features ...  464/1509
Extract Test Features ...  465/1509
Extract Test Features ...  466/1509
Extract Test Features ...  467/1509
Extract Test Features ...  468/1509
Extract Test Features ...  469/1509
Extract Test Features ...  470/1509
Extract Test Features ...  471/1509
Extract Test Features ...  472/1509
Extract Test Features ...  473/1509
Extract Test Features ...  474/1509
Extract Test Features ...  475/1509
Extract Test Features ...  476/1509
Extract Test Features ...  477/1509
Extract Test Features ...  478/1509
Extract Test Features ...  479/1509
Extract Test Features ...  480/1509
Extract Test Features ...  481/1509
Extract Test Features ...  482/1509
Extract Test Features ...  483/1509
Extract Test Features ...  484/1509
Extract Test Features ...  485/1509
Extract Test Features ...  486/1509
Extract Test Features ...  487/1509
Extract Test Features ...  488/1509
Extract Test Features ...  489/1509
Extract Test Features ...  4

Extract Test Features ...  698/1509
Extract Test Features ...  699/1509
Extract Test Features ...  700/1509
Extract Test Features ...  701/1509
Extract Test Features ...  702/1509
Extract Test Features ...  703/1509
Extract Test Features ...  704/1509
Extract Test Features ...  705/1509
Extract Test Features ...  706/1509
Extract Test Features ...  707/1509
Extract Test Features ...  708/1509
Extract Test Features ...  709/1509
Extract Test Features ...  710/1509
Extract Test Features ...  711/1509
Extract Test Features ...  712/1509
Extract Test Features ...  713/1509
Extract Test Features ...  714/1509
Extract Test Features ...  715/1509
Extract Test Features ...  716/1509
Extract Test Features ...  717/1509
Extract Test Features ...  718/1509
Extract Test Features ...  719/1509
Extract Test Features ...  720/1509
Extract Test Features ...  721/1509
Extract Test Features ...  722/1509
Extract Test Features ...  723/1509
Extract Test Features ...  724/1509
Extract Test Features ...  7

Extract Test Features ...  934/1509
Extract Test Features ...  935/1509
Extract Test Features ...  936/1509
Extract Test Features ...  937/1509
Extract Test Features ...  938/1509
Extract Test Features ...  939/1509
Extract Test Features ...  940/1509
Extract Test Features ...  941/1509
Extract Test Features ...  942/1509
Extract Test Features ...  943/1509
Extract Test Features ...  944/1509
Extract Test Features ...  945/1509
Extract Test Features ...  946/1509
Extract Test Features ...  947/1509
Extract Test Features ...  948/1509
Extract Test Features ...  949/1509
Extract Test Features ...  950/1509
Extract Test Features ...  951/1509
Extract Test Features ...  952/1509
Extract Test Features ...  953/1509
Extract Test Features ...  954/1509
Extract Test Features ...  955/1509
Extract Test Features ...  956/1509
Extract Test Features ...  957/1509
Extract Test Features ...  958/1509
Extract Test Features ...  959/1509
Extract Test Features ...  960/1509
Extract Test Features ...  9

Extract Test Features ... 1162/1509
Extract Test Features ... 1163/1509
Extract Test Features ... 1164/1509
Extract Test Features ... 1165/1509
Extract Test Features ... 1166/1509
Extract Test Features ... 1167/1509
Extract Test Features ... 1168/1509
Extract Test Features ... 1169/1509
Extract Test Features ... 1170/1509
Extract Test Features ... 1171/1509
Extract Test Features ... 1172/1509
Extract Test Features ... 1173/1509
Extract Test Features ... 1174/1509
Extract Test Features ... 1175/1509
Extract Test Features ... 1176/1509
Extract Test Features ... 1177/1509
Extract Test Features ... 1178/1509
Extract Test Features ... 1179/1509
Extract Test Features ... 1180/1509
Extract Test Features ... 1181/1509
Extract Test Features ... 1182/1509
Extract Test Features ... 1183/1509
Extract Test Features ... 1184/1509
Extract Test Features ... 1185/1509
Extract Test Features ... 1186/1509
Extract Test Features ... 1187/1509
Extract Test Features ... 1188/1509
Extract Test Features ... 11

Extract Test Features ... 1395/1509
Extract Test Features ... 1396/1509
Extract Test Features ... 1397/1509
Extract Test Features ... 1398/1509
Extract Test Features ... 1399/1509
Extract Test Features ... 1400/1509
Extract Test Features ... 1401/1509
Extract Test Features ... 1402/1509
Extract Test Features ... 1403/1509
Extract Test Features ... 1404/1509
Extract Test Features ... 1405/1509
Extract Test Features ... 1406/1509
Extract Test Features ... 1407/1509
Extract Test Features ... 1408/1509
Extract Test Features ... 1409/1509
Extract Test Features ... 1410/1509
Extract Test Features ... 1411/1509
Extract Test Features ... 1412/1509
Extract Test Features ... 1413/1509
Extract Test Features ... 1414/1509
Extract Test Features ... 1415/1509
Extract Test Features ... 1416/1509
Extract Test Features ... 1417/1509
Extract Test Features ... 1418/1509
Extract Test Features ... 1419/1509
Extract Test Features ... 1420/1509
Extract Test Features ... 1421/1509
Extract Test Features ... 14

In [21]:
'''
Normalizing
'''
flattened_train_features = np.concatenate(train_features, axis=0)
pca = PCA(n_components=flattened_train_features.shape[-1], whiten=True)
pca.fit(flattened_train_features)

PCA(n_components=128, whiten=True)

In [22]:
train_normalized_features = list()
index = 0

for features in train_features:
    features = pca.transform(features)
    train_normalized_features.append(features)
    index += 1
    print("Normalize Train Features ... {:4d}/{:4d}".format(index, len(train_features)))

Normalize Train Features ...    1/1538
Normalize Train Features ...    2/1538
Normalize Train Features ...    3/1538
Normalize Train Features ...    4/1538
Normalize Train Features ...    5/1538
Normalize Train Features ...    6/1538
Normalize Train Features ...    7/1538
Normalize Train Features ...    8/1538
Normalize Train Features ...    9/1538
Normalize Train Features ...   10/1538
Normalize Train Features ...   11/1538
Normalize Train Features ...   12/1538
Normalize Train Features ...   13/1538
Normalize Train Features ...   14/1538
Normalize Train Features ...   15/1538
Normalize Train Features ...   16/1538
Normalize Train Features ...   17/1538
Normalize Train Features ...   18/1538
Normalize Train Features ...   19/1538
Normalize Train Features ...   20/1538
Normalize Train Features ...   21/1538
Normalize Train Features ...   22/1538
Normalize Train Features ...   23/1538
Normalize Train Features ...   24/1538
Normalize Train Features ...   25/1538
Normalize Train Features 

Normalize Train Features ...  420/1538
Normalize Train Features ...  421/1538
Normalize Train Features ...  422/1538
Normalize Train Features ...  423/1538
Normalize Train Features ...  424/1538
Normalize Train Features ...  425/1538
Normalize Train Features ...  426/1538
Normalize Train Features ...  427/1538
Normalize Train Features ...  428/1538
Normalize Train Features ...  429/1538
Normalize Train Features ...  430/1538
Normalize Train Features ...  431/1538
Normalize Train Features ...  432/1538
Normalize Train Features ...  433/1538
Normalize Train Features ...  434/1538
Normalize Train Features ...  435/1538
Normalize Train Features ...  436/1538
Normalize Train Features ...  437/1538
Normalize Train Features ...  438/1538
Normalize Train Features ...  439/1538
Normalize Train Features ...  440/1538
Normalize Train Features ...  441/1538
Normalize Train Features ...  442/1538
Normalize Train Features ...  443/1538
Normalize Train Features ...  444/1538
Normalize Train Features 

Normalize Train Features ...  836/1538
Normalize Train Features ...  837/1538
Normalize Train Features ...  838/1538
Normalize Train Features ...  839/1538
Normalize Train Features ...  840/1538
Normalize Train Features ...  841/1538
Normalize Train Features ...  842/1538
Normalize Train Features ...  843/1538
Normalize Train Features ...  844/1538
Normalize Train Features ...  845/1538
Normalize Train Features ...  846/1538
Normalize Train Features ...  847/1538
Normalize Train Features ...  848/1538
Normalize Train Features ...  849/1538
Normalize Train Features ...  850/1538
Normalize Train Features ...  851/1538
Normalize Train Features ...  852/1538
Normalize Train Features ...  853/1538
Normalize Train Features ...  854/1538
Normalize Train Features ...  855/1538
Normalize Train Features ...  856/1538
Normalize Train Features ...  857/1538
Normalize Train Features ...  858/1538
Normalize Train Features ...  859/1538
Normalize Train Features ...  860/1538
Normalize Train Features 

Normalize Train Features ... 1258/1538
Normalize Train Features ... 1259/1538
Normalize Train Features ... 1260/1538
Normalize Train Features ... 1261/1538
Normalize Train Features ... 1262/1538
Normalize Train Features ... 1263/1538
Normalize Train Features ... 1264/1538
Normalize Train Features ... 1265/1538
Normalize Train Features ... 1266/1538
Normalize Train Features ... 1267/1538
Normalize Train Features ... 1268/1538
Normalize Train Features ... 1269/1538
Normalize Train Features ... 1270/1538
Normalize Train Features ... 1271/1538
Normalize Train Features ... 1272/1538
Normalize Train Features ... 1273/1538
Normalize Train Features ... 1274/1538
Normalize Train Features ... 1275/1538
Normalize Train Features ... 1276/1538
Normalize Train Features ... 1277/1538
Normalize Train Features ... 1278/1538
Normalize Train Features ... 1279/1538
Normalize Train Features ... 1280/1538
Normalize Train Features ... 1281/1538
Normalize Train Features ... 1282/1538
Normalize Train Features 

Normalize Train Features ... 1490/1538
Normalize Train Features ... 1491/1538
Normalize Train Features ... 1492/1538
Normalize Train Features ... 1493/1538
Normalize Train Features ... 1494/1538
Normalize Train Features ... 1495/1538
Normalize Train Features ... 1496/1538
Normalize Train Features ... 1497/1538
Normalize Train Features ... 1498/1538
Normalize Train Features ... 1499/1538
Normalize Train Features ... 1500/1538
Normalize Train Features ... 1501/1538
Normalize Train Features ... 1502/1538
Normalize Train Features ... 1503/1538
Normalize Train Features ... 1504/1538
Normalize Train Features ... 1505/1538
Normalize Train Features ... 1506/1538
Normalize Train Features ... 1507/1538
Normalize Train Features ... 1508/1538
Normalize Train Features ... 1509/1538
Normalize Train Features ... 1510/1538
Normalize Train Features ... 1511/1538
Normalize Train Features ... 1512/1538
Normalize Train Features ... 1513/1538
Normalize Train Features ... 1514/1538
Normalize Train Features 

In [23]:
test_normalized_features = list()
index = 0

for features in test_features:
    features = pca.transform(features)
    test_normalized_features.append(features)
    index += 1
    print("Normalize Test Features ... {:4d}/{:4d}".format(index, len(test_features)))

Normalize Test Features ...    1/1509
Normalize Test Features ...    2/1509
Normalize Test Features ...    3/1509
Normalize Test Features ...    4/1509
Normalize Test Features ...    5/1509
Normalize Test Features ...    6/1509
Normalize Test Features ...    7/1509
Normalize Test Features ...    8/1509
Normalize Test Features ...    9/1509
Normalize Test Features ...   10/1509
Normalize Test Features ...   11/1509
Normalize Test Features ...   12/1509
Normalize Test Features ...   13/1509
Normalize Test Features ...   14/1509
Normalize Test Features ...   15/1509
Normalize Test Features ...   16/1509
Normalize Test Features ...   17/1509
Normalize Test Features ...   18/1509
Normalize Test Features ...   19/1509
Normalize Test Features ...   20/1509
Normalize Test Features ...   21/1509
Normalize Test Features ...   22/1509
Normalize Test Features ...   23/1509
Normalize Test Features ...   24/1509
Normalize Test Features ...   25/1509
Normalize Test Features ...   26/1509
Normalize Te

Normalize Test Features ...  234/1509
Normalize Test Features ...  235/1509
Normalize Test Features ...  236/1509
Normalize Test Features ...  237/1509
Normalize Test Features ...  238/1509
Normalize Test Features ...  239/1509
Normalize Test Features ...  240/1509
Normalize Test Features ...  241/1509
Normalize Test Features ...  242/1509
Normalize Test Features ...  243/1509
Normalize Test Features ...  244/1509
Normalize Test Features ...  245/1509
Normalize Test Features ...  246/1509
Normalize Test Features ...  247/1509
Normalize Test Features ...  248/1509
Normalize Test Features ...  249/1509
Normalize Test Features ...  250/1509
Normalize Test Features ...  251/1509
Normalize Test Features ...  252/1509
Normalize Test Features ...  253/1509
Normalize Test Features ...  254/1509
Normalize Test Features ...  255/1509
Normalize Test Features ...  256/1509
Normalize Test Features ...  257/1509
Normalize Test Features ...  258/1509
Normalize Test Features ...  259/1509
Normalize Te

Normalize Test Features ...  450/1509
Normalize Test Features ...  451/1509
Normalize Test Features ...  452/1509
Normalize Test Features ...  453/1509
Normalize Test Features ...  454/1509
Normalize Test Features ...  455/1509
Normalize Test Features ...  456/1509
Normalize Test Features ...  457/1509
Normalize Test Features ...  458/1509
Normalize Test Features ...  459/1509
Normalize Test Features ...  460/1509
Normalize Test Features ...  461/1509
Normalize Test Features ...  462/1509
Normalize Test Features ...  463/1509
Normalize Test Features ...  464/1509
Normalize Test Features ...  465/1509
Normalize Test Features ...  466/1509
Normalize Test Features ...  467/1509
Normalize Test Features ...  468/1509
Normalize Test Features ...  469/1509
Normalize Test Features ...  470/1509
Normalize Test Features ...  471/1509
Normalize Test Features ...  472/1509
Normalize Test Features ...  473/1509
Normalize Test Features ...  474/1509
Normalize Test Features ...  475/1509
Normalize Te

Normalize Test Features ...  828/1509
Normalize Test Features ...  829/1509
Normalize Test Features ...  830/1509
Normalize Test Features ...  831/1509
Normalize Test Features ...  832/1509
Normalize Test Features ...  833/1509
Normalize Test Features ...  834/1509
Normalize Test Features ...  835/1509
Normalize Test Features ...  836/1509
Normalize Test Features ...  837/1509
Normalize Test Features ...  838/1509
Normalize Test Features ...  839/1509
Normalize Test Features ...  840/1509
Normalize Test Features ...  841/1509
Normalize Test Features ...  842/1509
Normalize Test Features ...  843/1509
Normalize Test Features ...  844/1509
Normalize Test Features ...  845/1509
Normalize Test Features ...  846/1509
Normalize Test Features ...  847/1509
Normalize Test Features ...  848/1509
Normalize Test Features ...  849/1509
Normalize Test Features ...  850/1509
Normalize Test Features ...  851/1509
Normalize Test Features ...  852/1509
Normalize Test Features ...  853/1509
Normalize Te

Normalize Test Features ... 1093/1509
Normalize Test Features ... 1094/1509
Normalize Test Features ... 1095/1509
Normalize Test Features ... 1096/1509
Normalize Test Features ... 1097/1509
Normalize Test Features ... 1098/1509
Normalize Test Features ... 1099/1509
Normalize Test Features ... 1100/1509
Normalize Test Features ... 1101/1509
Normalize Test Features ... 1102/1509
Normalize Test Features ... 1103/1509
Normalize Test Features ... 1104/1509
Normalize Test Features ... 1105/1509
Normalize Test Features ... 1106/1509
Normalize Test Features ... 1107/1509
Normalize Test Features ... 1108/1509
Normalize Test Features ... 1109/1509
Normalize Test Features ... 1110/1509
Normalize Test Features ... 1111/1509
Normalize Test Features ... 1112/1509
Normalize Test Features ... 1113/1509
Normalize Test Features ... 1114/1509
Normalize Test Features ... 1115/1509
Normalize Test Features ... 1116/1509
Normalize Test Features ... 1117/1509
Normalize Test Features ... 1118/1509
Normalize Te

Normalize Test Features ... 1391/1509
Normalize Test Features ... 1392/1509
Normalize Test Features ... 1393/1509
Normalize Test Features ... 1394/1509
Normalize Test Features ... 1395/1509
Normalize Test Features ... 1396/1509
Normalize Test Features ... 1397/1509
Normalize Test Features ... 1398/1509
Normalize Test Features ... 1399/1509
Normalize Test Features ... 1400/1509
Normalize Test Features ... 1401/1509
Normalize Test Features ... 1402/1509
Normalize Test Features ... 1403/1509
Normalize Test Features ... 1404/1509
Normalize Test Features ... 1405/1509
Normalize Test Features ... 1406/1509
Normalize Test Features ... 1407/1509
Normalize Test Features ... 1408/1509
Normalize Test Features ... 1409/1509
Normalize Test Features ... 1410/1509
Normalize Test Features ... 1411/1509
Normalize Test Features ... 1412/1509
Normalize Test Features ... 1413/1509
Normalize Test Features ... 1414/1509
Normalize Test Features ... 1415/1509
Normalize Test Features ... 1416/1509
Normalize Te

[문제 2. Bag-of-Features 구현 – 30 pts]  

2-1.   
K-means을 통해 구해진 codebook을 통하여, 각각의 이미지의 특징을 인코딩(histogram화 혹은  양자화 라고도 함) 하시오.  
(자세한 사항은 템플릿 코드를 참고)  

In [ ]:
'''
Make Codebook
'''

In [41]:
class Codebook:

    def __init__(self, K):

        self.K = K
        
        self.kmeans = KMeans(n_clusters=K, verbose=True)

    def make_code_words(self, features):

        self.kmeans.fit(features)

    def encode(self, features, shapes):

        distances = self.kmeans.transform(features) #centroids, codewords와 각 이미지 특징들 간의 거리
        
        print(distances)
        # reshaped_features는 Spatial Pyramid Matching 문제에 활용하세요.
        reshaped_features = np.reshape(features, (shapes[0], shapes[1], -1))

        # 1) 함수 encode 부분 안의 None 부분을 채우세요.
        #    distances는 K means 알고리즘을 통해 얻어진 centroids, 즉 codewords(visual words)와 각 이미지의 특징들 간의 거리 입니다.
        #    distances 값을 이용하여, features(# of keypoints, feature_dim)를 인코딩(histogram 혹은 quantization이라고도 함) 하세요.
        #    인코딩된 결과인 representations은 (K)로 표현되어야 합니다. 이 때, K는 codewords의 개수입니다.
        
        #distances.cdist()
        idx = np.argmin(distances, axis = 0)
        hist, bin_edges = np.histogram(idx, bins=50) # 50은 왜?
        print(hist)
        #hist_norm = [float/sum(hist)]
        
        representations = hist

        if np.array(representations).shape != (self.K, ):
            # representations는 반드시 (K) 차원을 가져야 합니다 (Spatial Pyramid Matching 사용 안할 시에만).
            print("Your code may be wrong") # assert 써볼까?

        return representations

In [42]:
flattened_normalized_train_features = pca.transform(flattened_train_features)
codebook = Codebook(K)
codebook.make_code_words(flattened_normalized_train_features)

Initialization complete
Iteration 0, inertia 130711432.0
Iteration 1, inertia 95996592.0
Iteration 2, inertia 94657512.0
Iteration 3, inertia 94030128.0
Iteration 4, inertia 93635296.0
Iteration 5, inertia 93345464.0
Iteration 6, inertia 93132120.0
Iteration 7, inertia 93016952.0
Iteration 8, inertia 92944736.0
Iteration 9, inertia 92887568.0
Iteration 10, inertia 92845408.0
Iteration 11, inertia 92819744.0
Iteration 12, inertia 92798992.0
Iteration 13, inertia 92778320.0
Iteration 14, inertia 92761488.0
Iteration 15, inertia 92745280.0
Iteration 16, inertia 92728680.0
Iteration 17, inertia 92710248.0
Iteration 18, inertia 92690544.0
Iteration 19, inertia 92670200.0
Iteration 20, inertia 92652536.0
Iteration 21, inertia 92635112.0
Iteration 22, inertia 92622696.0
Iteration 23, inertia 92614864.0
Iteration 24, inertia 92609016.0
Iteration 25, inertia 92605896.0
Iteration 26, inertia 92603776.0
Iteration 27, inertia 92601904.0
Iteration 28, inertia 92598992.0
Iteration 29, inertia 925967

Iteration 29, inertia 92730992.0
Iteration 30, inertia 92728152.0
Iteration 31, inertia 92725192.0
Iteration 32, inertia 92722096.0
Iteration 33, inertia 92719320.0
Iteration 34, inertia 92715280.0
Iteration 35, inertia 92711096.0
Iteration 36, inertia 92705280.0
Iteration 37, inertia 92698576.0
Iteration 38, inertia 92693504.0
Iteration 39, inertia 92689384.0
Iteration 40, inertia 92686952.0
Iteration 41, inertia 92684432.0
Iteration 42, inertia 92682456.0
Iteration 43, inertia 92680824.0
Iteration 44, inertia 92679776.0
Iteration 45, inertia 92679328.0
Iteration 46, inertia 92679728.0
Iteration 47, inertia 92679176.0
Iteration 48, inertia 92679392.0
Iteration 49, inertia 92679352.0
Iteration 50, inertia 92679208.0
Iteration 51, inertia 92679728.0
Iteration 52, inertia 92679416.0
Iteration 53, inertia 92679576.0
Iteration 54, inertia 92679680.0
Iteration 55, inertia 92679752.0
Iteration 56, inertia 92679784.0
Iteration 57, inertia 92679560.0
Iteration 58, inertia 92679264.0
Iteration 

Iteration 64, inertia 92565520.0
Iteration 65, inertia 92565528.0
Iteration 66, inertia 92565392.0
Iteration 67, inertia 92565400.0
Iteration 68, inertia 92565248.0
Iteration 69, inertia 92565304.0
Converged at iteration 69: center shift 8.975119999377057e-05 within tolerance 9.99611496925354e-05.
Initialization complete
Iteration 0, inertia 125947800.0
Iteration 1, inertia 95868800.0
Iteration 2, inertia 94570232.0
Iteration 3, inertia 93987856.0
Iteration 4, inertia 93610216.0
Iteration 5, inertia 93332688.0
Iteration 6, inertia 93136432.0
Iteration 7, inertia 93027072.0
Iteration 8, inertia 92971280.0
Iteration 9, inertia 92936272.0
Iteration 10, inertia 92909672.0
Iteration 11, inertia 92887488.0
Iteration 12, inertia 92866488.0
Iteration 13, inertia 92845352.0
Iteration 14, inertia 92825776.0
Iteration 15, inertia 92802768.0
Iteration 16, inertia 92768872.0
Iteration 17, inertia 92733200.0
Iteration 18, inertia 92719352.0
Iteration 19, inertia 92713304.0
Iteration 20, inertia 9270

Iteration 52, inertia 92548104.0
Iteration 53, inertia 92547760.0
Iteration 54, inertia 92547024.0
Iteration 55, inertia 92546824.0
Iteration 56, inertia 92547016.0
Iteration 57, inertia 92546872.0
Iteration 58, inertia 92546488.0
Iteration 59, inertia 92546496.0
Iteration 60, inertia 92546888.0
Iteration 61, inertia 92546592.0
Iteration 62, inertia 92546648.0
Iteration 63, inertia 92546416.0
Iteration 64, inertia 92545872.0
Iteration 65, inertia 92545792.0
Iteration 66, inertia 92545624.0
Iteration 67, inertia 92545512.0
Iteration 68, inertia 92545472.0
Iteration 69, inertia 92545480.0
Iteration 70, inertia 92545424.0
Iteration 71, inertia 92545544.0
Iteration 72, inertia 92545392.0
Iteration 73, inertia 92545040.0
Iteration 74, inertia 92545064.0
Iteration 75, inertia 92544944.0
Iteration 76, inertia 92544568.0
Iteration 77, inertia 92544224.0
Iteration 78, inertia 92543304.0
Iteration 79, inertia 92542848.0
Iteration 80, inertia 92542376.0
Iteration 81, inertia 92541128.0
Iteration 

Iteration 43, inertia 92573456.0
Iteration 44, inertia 92573208.0
Iteration 45, inertia 92573104.0
Iteration 46, inertia 92573072.0
Iteration 47, inertia 92573000.0
Iteration 48, inertia 92573072.0
Iteration 49, inertia 92573368.0
Iteration 50, inertia 92573384.0
Iteration 51, inertia 92573352.0
Iteration 52, inertia 92573224.0
Iteration 53, inertia 92573040.0
Iteration 54, inertia 92573072.0
Iteration 55, inertia 92573152.0
Iteration 56, inertia 92573344.0
Iteration 57, inertia 92573584.0
Iteration 58, inertia 92573600.0
Iteration 59, inertia 92573576.0
Converged at iteration 59: center shift 9.087578655453399e-05 within tolerance 9.99611496925354e-05.


codebook = Codebook(k)에서는 initalize만 수행했으므로 encode의 내용을 수정한다 해서 ln[26]을 또 수행할 필요는 없다.

In [43]:
'''
Encode Features
'''
train_encoded_features = []
index = 0

for features, shapes in zip(train_normalized_features, train_feature_shapes):
    encoded_features = codebook.encode(features, shapes)
    train_encoded_features.append(encoded_features)
    index += 1
    print("Encoding Train Features ... {:4d}/{:4d}".format(index, len(train_normalized_features)))

[[14.436974 14.498974 13.971121 ... 14.313533 14.637489 14.440004]
 [16.745598 16.964842 16.606705 ... 16.796202 17.01693  16.582003]
 [14.99374  15.070861 14.641543 ... 14.337738 14.904948 14.939386]
 ...
 [13.355696 13.222261 12.968501 ... 13.114889 11.798833 13.378341]
 [14.682595 14.538004 14.30608  ... 14.499327 14.684841 14.609341]
 [12.430556 12.720158 12.23946  ... 12.604323 12.760173 12.665977]]
[ 1  0  0  0  0  0  0  0  0  0  0  0  1  1 16  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  1]
Your code may be wrong
Encoding Train Features ...    1/1538
[[13.3227    13.263497  12.897556  ... 12.664     13.417734  13.1923065]
 [13.106825  12.983922  12.655758  ... 12.800558  12.654301  12.218125 ]
 [13.341044  13.098221  12.871135  ... 12.128483  13.394346  13.192164 ]
 ...
 [ 7.6893954  7.7522917  6.370293  ...  7.3500957  7.6018095  7.7033186]
 [ 8.76006    8.6238785  7.3798466 ...  8.4388685  8.632749   8.692772 ]
 [12.88

ValueError: cannot reshape array of size 31360 into shape (16,16,newaxis)

In [ ]:
test_encoded_features = []
index = 0

for features, shapes in zip(test_normalized_features, test_feature_shapes):
    encoded_features = codebook.encode(features, shapes)
    test_encoded_features.append(encoded_features)
    index += 1
    print("Encoding Text Features ... {:4d}/{:4d}".format(index, len(test_normalized_features)))

[문제 3. SVM을 통한 이미지 분류 – 15 pts]  

3-1.   
Bag-of-Features 알고리즘을 통해 얻어진 인코딩된 벡터를 통해, SVM을 학습하시오.  
이 때, sklearn과 같은 패키지를 활용하여 SVM 학습을 구현하시면 됩니다. (자세한 사항은 템플릿 코드를 참고)

In [ ]:
'''
Approximate Kernel
'''
chi2sampler = AdditiveChi2Sampler(sample_steps=2)
chi2sampler.fit(train_encoded_features, y_train)
train_encoded_features = chi2sampler.transform(train_encoded_features)
test_encoded_features = chi2sampler.transform(test_encoded_features)

In [ ]:
'''
Classify Images with SVM
'''

start_time = time.time()

# 1) 아래의 model 부분에 sklearn 패키지를 활용하여, Linear SVM(SVC) 모델을 정의하세요.
#    처음에는 SVM의 parameter를 기본으로 설정하여 구동하시길 권장합니다.
#    구동 성공 시, SVM의 C 값과 max_iter 파라미터 등을 조정하여 성능 향상을 해보시길 바랍니다.
model = LinearSVC(C=700.0,
                  class_weight=None, dual=True,
                  fit_intercept=True,
                  intercept_scaling=1,
                  loss='squared_hinge',
                  max_iter= 2000, multi_class='ovr',
                  penalty='l2', random_state=0, tol= 1e-4,
                    verbose=0)

print("Classify Images ...")
model.fit(train_encoded_features, y_train)
train_score = model.score(train_encoded_features, y_train)
test_score = model.score(test_encoded_features, y_test)

elapsed_time = time.time() - start_time

In [ ]:
'''
Print Results
'''
print()
print("=" * 90)
print("Train  Score: {:.5f}".format(train_score))
print("Test   Score: {:.5f}".format(test_score))
print("Elapsed Time: {:.2f} secs".format(elapsed_time))
print("=" * 90)

[문제 4. 파라미터 조정을 통한 성능 개선 – 40 pts] 

4-1.   
현재 주어진 코드에는 여러 하이퍼 파라미터가 존재합니다.    
예를 들어, Keypoint를 만들 때의 간격(patch_stride), codebook의 visual word 개수(K), SVM의 학습 파라미터 (C 값 및 max_iter) 등 이 있습니다.   
해당 파라미터들을 수정해가면서 결과를 개선시켜 보시오. (K와 patch_stride 같은 경우, 메모리 부족으로 큰 데이터를 사용하지 못 할 수 있으므로, 절대적인 성능 지표보다, 상대적인 성능 향상을 더 고려할 예정임)  

4-2.  
앞서 여러 파라미터의 실험 결과를 토대로 개선된 혹은 개선 되지 않은 이유를 보고서에 설명하시오.   
(예를 들어, 각 파라미터 별로 결과 테이블 정리 및 결과에 대한 이유 등).

[심화 문제 5. 알고리즘 개선 – 가산점 +20 pts]  

(본 문제는 풀이를 하지 않으셔도 감점되지 않습니다.)  
5-1. Bag-of-Features 알고리즘의 단점 중 하나는 공간의 배열, 위치 관계를 보지 않고 특징을 인코딩한다는 점입니다.   
즉, 특정 특징들의 개수 만을 통해서 인코딩을 하기 때문에 물체의 위치, 배열 등의 패턴 차이는 제대로 표현되지 않을 수 있습니다.   
이러한 단점을 극복하기 위하여 Spatial Pyramid Matching (공간 분할)을 수행 후, codewords을 통해 인코딩 할 수 있습니다.   
앞서 언급된 Spatial Pyramid Matching 기법을 적용하여 성능을 개선 시켜 보시오.  
(https://slazebni.cs.illinois.edu/publications/pyramid_chapter.pdf를 참고하시면 도움이 됩니다.)